### Import pandas and read in the csv file and set it to a dataframe called baskets

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

### put the utility functions together

In [2]:
def add_date_cols(baskets):
    baskets['datetime'] = pd.to_datetime(baskets['placed_at'])
    baskets['year'] = baskets["datetime"].dt.year
    baskets['month'] = baskets["datetime"].dt.month
    baskets['date'] = baskets["datetime"].dt.date
    baskets['day'] = baskets["datetime"].dt.day
    baskets['hour'] = baskets["datetime"].dt.hour
    baskets['weekday'] = baskets["datetime"].dt.weekday
    baskets['year_month'] = baskets["datetime"].apply(lambda t: t.strftime("%Y-%m"))
    baskets['month_num'] = (baskets['year'] - 2021) * 12 + baskets['month']
    baskets['year_week'] = baskets["datetime"].apply(lambda t: t.strftime("%Y-%W")) # this makes the beginning of Jan 2022 as week 2022-00 , not 2022-52
    baskets['week_num'] = baskets["datetime"].apply(lambda t: int(t.strftime("%W"))) 
    baskets['iso_week_num'] = baskets["datetime"].dt.isocalendar().week # this returns week number 52 for Jan 1, 2021, not 0 which is what we want
    baskets['cum_week_num'] = (baskets['year'] - 2021) * 52 + baskets['week_num']
    return baskets
def get_merchant_attributes(baskets):
    merchant_attributes = baskets.groupby(['merchant_id']).agg(
        total_spent = ('spent', 'sum'), 
        num_orders = ('order_id', 'nunique'), 
        first_month = ('month_num', 'min'), 
        last_month = ('month_num', 'max'), 
        num_months = ('month_num', 'nunique'), 
        num_weeks = ('week_num', 'nunique'), 
        num_days = ('date', 'nunique'), 
        num_skus = ('sku_id','nunique'), 
        num_top_cats = ('top_cat','nunique'), 
        num_sub_cats = ('sub_cat','nunique'),
    ).reset_index()
    merchant_attributes['avg_spent_per_order'] = merchant_attributes.total_spent / merchant_attributes.num_orders
    merchant_attributes['tenure_month'] = merchant_attributes.last_month - merchant_attributes.first_month +1
    return merchant_attributes

In [3]:
filename = './AwanTunai_transaction_data.csv'
baskets = pd.read_csv(filename)
baskets["spent"] = baskets["qty"] * baskets["price"] # column wide multiplication
baskets = add_date_cols(baskets)
merchant_attributes = get_merchant_attributes(baskets)

### Questions

 - which orders happened on the exact same millisecond

In [4]:
baskets.nunique()

id              492437
order_id         53438
placed_at        53434
merchant_id        357
sku_id            1860
top_cat             35
sub_cat            108
qty                 97
price             1607
spent             3289
datetime         53434
year                 2
month               12
date               544
day                 31
hour                24
weekday              7
year_month          19
month_num           19
year_week           82
week_num            53
iso_week_num        52
cum_week_num        81
dtype: int64

 - If we take unique count of baskets' attributes, we notice that unique order_id is greater than unique placed_at, by 4. 
 - noteice that "placed_at" is in milisecond, is it possible that some orders are made on exactly the same milisecond? In theory it is possible, but might there be potential fraud?
  - how can we find out which orders happened on the exact same millisecond? 

 - why are the count on top_cat_id and sub_cat_id different from others? 
 - why would some items have price of 1?

 - we only need to look at columns of order_id and placed_at, and find which rows have different order_id but same placed_at time
 - any rows that have exact same order_id and exactly same placed_at are considered duplicate and we can drop these rows

 - use drop_duplicates , default parameter is keep='first', i.e., among each group of duplicates, only keep the first duplicate
 - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html

In [13]:
df.order_id.nunique()

53438

 - and then look at all rows with same placed_at, find the duplicates 
 - the order_id's in those rows would be the orders that have antoher order that have the same placed_at time

In [23]:
df = baskets.drop_duplicates(subset = ['order_id','placed_at'])
o_id = df[df.duplicated(subset = ['placed_at'],keep=False)]['order_id'].reset_index()

In [24]:
o_id

,index,order_id
0,43236,4491
1,43237,4490
2,48342,4960
3,48343,4961
4,54728,5540
5,54729,5539
6,194561,18724
7,194562,18725


In [16]:
baskets[baskets['order_id'].isin(o_id['order_id'])][["order_id","placed_at"]].drop_duplicates()

,order_id,placed_at
43236,4491,2021-10-12 17:25:54.920
43237,4490,2021-10-12 17:25:54.920
48342,4960,2021-10-18 08:36:36.648
48343,4961,2021-10-18 08:36:36.648
54728,5540,2021-10-24 08:51:09.004
54729,5539,2021-10-24 08:51:09.004
194561,18724,2022-03-16 17:14:52.083
194562,18725,2022-03-16 17:14:52.083


### Question
 - do the same SKUs within the same order have the same price?

- by counting the unique number of prices within the same order on the same sku
baskets.groupby(["order_id","sku_id"]).price.nunique().sort_values(ascending=False).head(10)
 - this says there are such cases, but only 7 such orders, and only 2 distict price values for the same sku

In [7]:
od = baskets.groupby(["order_id","sku_id"]).price.nunique().reset_index()
odd = od[od.price > 1]
odd

,order_id,sku_id,price
231379,22691,1054,2
258365,25707,1051,2
337754,34868,874,2
339529,35098,874,2
340946,35285,1504,2
343731,35648,874,2
346017,35916,1504,2


 - merge back to baskets to locate the full transaction records

In [8]:
baskets.merge(odd, left_on=["order_id", "sku_id"],right_on=["order_id", "sku_id"])

,id,order_id,placed_at,merchant_id,sku_id,top_cat,sub_cat,qty,price_x,spent,...,day,hour,weekday,year_month,month_num,year_week,week_num,iso_week_num,cum_week_num,price_y
0,231128,22691,2022-04-22 10:45:03.279,280,1054,10.0,45.0,1,78000.0,78000.0,...,22,10,4,2022-04,16,2022-16,16,16,68,2
1,231129,22691,2022-04-22 10:45:03.279,280,1054,10.0,45.0,1,104000.0,104000.0,...,22,10,4,2022-04,16,2022-16,16,16,68,2
2,258281,25707,2022-05-25 11:32:51.909,283,1051,10.0,45.0,1,89000.0,89000.0,...,25,11,2,2022-05,17,2022-21,21,21,73,2
3,258282,25707,2022-05-25 11:32:51.909,283,1051,10.0,45.0,1,99000.0,99000.0,...,25,11,2,2022-05,17,2022-21,21,21,73,2
4,339135,34868,2022-08-11 08:16:00.675,68,874,27.0,86.0,1,173000.0,173000.0,...,11,8,3,2022-08,20,2022-32,32,32,84,2
5,339136,34868,2022-08-11 08:16:00.675,68,874,27.0,86.0,1,174000.0,174000.0,...,11,8,3,2022-08,20,2022-32,32,32,84,2
6,340344,35098,2022-08-12 11:16:11.746,68,874,27.0,86.0,1,173000.0,173000.0,...,12,11,4,2022-08,20,2022-32,32,32,84,2
7,340345,35098,2022-08-12 11:16:11.746,68,874,27.0,86.0,2,174000.0,348000.0,...,12,11,4,2022-08,20,2022-32,32,32,84,2
8,340890,35285,2022-08-12 17:07:46.285,236,1504,27.0,86.0,1,252500.0,252500.0,...,12,17,4,2022-08,20,2022-32,32,32,84,2
9,340891,35285,2022-08-12 17:07:46.285,236,1504,27.0,86.0,1,257000.0,257000.0,...,12,17,4,2022-08,20,2022-32,32,32,84,2


 - looking at the orders, including all skus in the orders

In [9]:
baskets.merge(odd, left_on=["order_id"],right_on=["order_id"])

,id,order_id,placed_at,merchant_id,sku_id_x,top_cat,sub_cat,qty,price_x,spent,...,hour,weekday,year_month,month_num,year_week,week_num,iso_week_num,cum_week_num,sku_id_y,price_y
0,231127,22691,2022-04-22 10:45:03.279,280,1059,10.0,45.0,1,74250.0,74250.0,...,10,4,2022-04,16,2022-16,16,16,68,1054,2
1,231128,22691,2022-04-22 10:45:03.279,280,1054,10.0,45.0,1,78000.0,78000.0,...,10,4,2022-04,16,2022-16,16,16,68,1054,2
2,231129,22691,2022-04-22 10:45:03.279,280,1054,10.0,45.0,1,104000.0,104000.0,...,10,4,2022-04,16,2022-16,16,16,68,1054,2
3,231317,22691,2022-04-22 10:45:03.279,280,973,27.0,86.0,3,71500.0,214500.0,...,10,4,2022-04,16,2022-16,16,16,68,1054,2
4,231318,22691,2022-04-22 10:45:03.279,280,1501,27.0,86.0,2,181000.0,362000.0,...,10,4,2022-04,16,2022-16,16,16,68,1054,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,351625,35916,2022-08-23 19:04:13.531,316,880,25.0,86.0,5,18400.0,92000.0,...,19,1,2022-08,20,2022-34,34,34,86,1504,2
71,351626,35916,2022-08-23 19:04:13.531,316,971,27.0,86.0,5,19800.0,99000.0,...,19,1,2022-08,20,2022-34,34,34,86,1504,2
72,351627,35916,2022-08-23 19:04:13.531,316,818,27.0,86.0,3,22300.0,66900.0,...,19,1,2022-08,20,2022-34,34,34,86,1504,2
73,351628,35916,2022-08-23 19:04:13.531,316,984,27.0,86.0,4,17800.0,71200.0,...,19,1,2022-08,20,2022-34,34,34,86,1504,2
